<p align="center">
  <img src="image_logo.png" alt="Logo du projet" style="width: 150px; height: auto; border-radius: 8px; box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);">
</p>

<h1 align="center" style="color: #FFA500; font-size: 2.5em; font-weight: bold;">Projet INS_PHAS</h1>

<p align="center" style="color: #4CAF50; font-size: 1.2em;">
  Collecte des données de prix au moyen des données scrapées et des données scannées.
</p>

<p align="center" style="color: #4CAF50; font-size: 1.1em;">
  <strong>Auteur :</strong> DOUMBIA ABDOULAYE (<a href="mailto:abdoulaye.doumbi19@inphb.ci" style="color: #FFA500;">abdoulaye.doumbi19@inphb.ci</a>)
</p>

<p align="center" style="color: #4CAF50; font-size: 1.1em;">
  Ce matériel est soumis aux termes et conditions de la licence <a href="https://creativecommons.org/licenses/by-nc-sa/4.0/" style="color: #FFA500;">Creative Commons CC BY-NC-SA 4.0</a>. L'utilisation gratuite est autorisée à des fins non commerciales.
</p>


# ivoirshop

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import requests
from urllib.parse import urljoin
import pandas as pd
import time
import re
import requests
from selenium.common.exceptions import WebDriverException

# Liste des URLs à traiter
urls = ["https://www.ivoirshop.ci/",
        "https://www.ivoirshop.ci/categorie-produit/supermache",
        "https://www.ivoirshop.ci/categorie-produit/maison-bureau",
        "https://www.ivoirshop.ci/categorie-produit/telephonie",
        "https://www.ivoirshop.ci/categorie-produit/beaute-hygiene",
        "https://www.ivoirshop.ci/categorie-produit/electronique",
        "https://www.ivoirshop.ci/categorie-produit/produits-adultes",
        "https://www.ivoirshop.ci/categorie-produit/mode/mode-femme",
        "https://www.ivoirshop.ci/categorie-produit/mode/mode-homme",
        "https://www.ivoirshop.ci/categorie-produit/produits-pour-bebes",
        "https://www.ivoirshop.ci/categorie-produit/informatique",
        "https://www.ivoirshop.ci/categorie-produit/sport-bien-etre",
        "https://www.ivoirshop.ci/categorie-produit/jouets-et-jeux-videos"
       ]

# Liste pour stocker les DataFrames de chaque site
all_dfs = []
products = []
# Boucle à travers chaque URL
for url in urls:
    # Étape 1 : Envoyer une requête HTTP à la page web et récupérer le contenu HTML
    response = requests.get(url)
    html_content = response.text

    # Étape 2 : Utiliser BeautifulSoup pour parser le contenu HTML
    soup = BeautifulSoup(html_content, "html.parser")

    # Étape 3 : Extraire les informations des produits
    
    for product_tag in soup.find_all("li", class_="product"):
        product_info = {}
        
        product_info['N_ordre']=""
        product_info["Code_ID_PE"]=""
        
        product_info['code_site']= url
        product_info['date de collecte']= datetime.now().strftime('%Y-%m-%d')

        # Extraire le lien du produit
        product_info["lien_produit"] = product_tag.find("a")["href"]

        # Extraire le titre du produit
        product_info["Libellé du produit"] = product_tag.find("h2", class_="woo-loop-product__title").text.strip()

        # Extraire le prix du produit
        price_tag = product_tag.find("span", class_="price")
        product_info["Prix du produit"] = price_tag.find("ins").text.strip() if price_tag and price_tag.find("ins") else None

        # Ajouter les informations du produit à la liste
        products.append(product_info)

    # Étape 4 : Créer un DataFrame à partir de la liste de produits
    df_ivoirshop = pd.DataFrame(products)

    # Ajouter le DataFrame à la liste globale
    all_dfs.append(df_ivoirshop)
df_ivoirshop

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_product_info(url):
    try:
        response = requests.get(url, timeout=500)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Erreur de connexion à {url} : {e}")
        return pd.DataFrame()

    soup = BeautifulSoup(response.text, 'html.parser')

    price_tag = soup.find('p', class_='price')
    promo_price_tag = price_tag.find('ins')
    promo_price = promo_price_tag.text.strip() if promo_price_tag else None

    regular_price_tag = price_tag.find('del')
    regular_price = regular_price_tag.text.strip() if regular_price_tag else None

    stock_tag = soup.find('p', class_='stock in-stock')
    stock = stock_tag.text.strip() if stock_tag else None

    description_tag = soup.find('h1', class_='wt-text-body-03')
    description = description_tag.text.strip() if description_tag else None
    
    url_lien = url

    # Create a DataFrame with the extracted information
    df_product = pd.DataFrame({
        
        'Prix Réel': [promo_price],
        'code_siteDescription': url_lien,
        'Quantité': [stock],
        'Caractéristiques du produit': [description]
    })

    return df_product

# List of URLs for individual product pages
urls = list(df_ivoirshop["lien_produit"])
# Initialize an empty DataFrame to store the results
combined_df_ivoirshop = pd.DataFrame()

# Scrape product information for each URL and concatenate the results
for url in urls:
    df_product = scrape_product_info(url)
    combined_df_ivoirshop = pd.concat([combined_df_ivoirshop, df_product], ignore_index=True)

# Display the combined DataFrame

combined_df_ivoirshop

In [ ]:
df_ivoirshop['Prix Réel'] = combined_df_ivoirshop['Prix Réel']
df_ivoirshop['code_siteDescription'] = combined_df_ivoirshop['code_siteDescription'] 
df_ivoirshop['Quantité'] = combined_df_ivoirshop['Quantité']
df_ivoirshop['Caractéristiques du produit'] = combined_df_ivoirshop['Caractéristiques du produit']
df_ivoirshop

In [ ]:
# Réorganiser les colonnes selon vos besoins
df_ivoirshop = df_ivoirshop[[
    'N_ordre', 'code_site', 'Code_ID_PE', 'date de collecte', 'Libellé du produit', 'Caractéristiques du produit',
    'Prix Réel', 'Quantité', "Prix du produit"]]
df_ivoirshop

# kevajo

In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_product_info(product_div):
    title_tag = product_div.find('h3', class_='wd-entities-title')
    title = title_tag.text.strip() if title_tag else None

    price_tag = product_div.find('span', class_='price')
    price = price_tag.text.strip() if price_tag else None

    promo_tag = product_div.find('span', class_='woocommerce-Price-amount amount')
    promo = promo_tag.text.strip() if promo_tag else None

    real_price_tag = product_div.find('ins', class_='woocommerce-Price-amount amount')
    real_price = real_price_tag.text.strip() if real_price_tag else None

    image_tag = product_div.find('img', class_='attachment-600x498')
    image_url = image_tag['nitro-lazy-src'] if image_tag and 'nitro-lazy-src' in image_tag.attrs else None

    label_tag = product_div.find('span', class_='awl-inner-text')
    label = label_tag.text.strip() if label_tag else None
    
    
    brand_tag = product_div.find("div", class_="col-12 mt-1 my-md-3 text-center text-md-start jt-max-line-size-3")
    brand = brand_tag.text.strip() if brand_tag else None
    
    
    
    quantity_tag = product_div.find('input', class_='js-item-qty')
    quantity = quantity_tag['value'] if quantity_tag else None

    product_url = product_div.find('a', class_='product-image-link')['href']
    date_new=datetime.now().strftime('%Y-%m-%d')

    return {
        'date de collecte': date_new,
        'Libellé du produit': title,
        'Prix du produit': price,
        'Promo': promo,
        'Image URL': image_url,
        'URL': product_url
    }

products_data = []
def scrape_page(url):
    try:
        response = requests.get(url, timeout=500)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Erreur de connexion à {url} : {e}")
        return pd.DataFrame()

    soup = BeautifulSoup(response.text, 'html.parser')

    product_divs = soup.find_all('div', class_='wd-product')
    
    
    for product_div in product_divs:
        product_info = scrape_product_info(product_div)
        products_data.append(product_info)

    return pd.DataFrame(products_data)

# URL de la page
urls = [
    "https://kevajo.com/", "https://kevajo.com/product-category/pour-bebe/",
    "https://kevajo.com/product-category/mode-2/modefemme/",
"https://kevajo.com/product-category/mode-2/mode-homme/",
"https://kevajo.com/product-category/maison-et-cuisine/",
"https://kevajo.com/product-category/fournitures-de-bureau-et-scolaires/",
"https://kevajo.com/product-category/telephones-et-tablettes/",
"https://kevajo.com/product-category/jeux-video-consoles-et-accessoires/",
"https://kevajo.com/product-category/electronique/",
"https://kevajo.com/product-category/lunettes-de-vue/",
"https://kevajo.com/product-category/beaute-et-hygiene/",
"https://kevajo.com/product-category/informatique/",
"https://kevajo.com/product-category/auto-et-moto/",
"https://kevajo.com/product-category/mode-2/bagages-et-sacs-de-voyage/",
"https://kevajo.com/#"]

# Scrape product information for each URL and concatenate the results
for url in urls:
    df_product = scrape_page(url)
    df_kevajo = pd.concat([ df_product], ignore_index=True)
df_kevajo




,date de collecte,Libellé du produit,Prix du produit,Promo,Image URL,URL
0,2024-01-26,Rouge à Lèvres Liquide Mat Et Crayon à Lèvres ...,"2,000 CFA","2,000 CFA",https://cdn-jgmnn.nitrocdn.com/IeLUGrClUJaCmTO...,https://kevajo.com/boutique/beaute-et-hygiene/...
1,2024-01-26,SLIM EXTREME 4D – Sérum Surconcentré Réducteur...,"9,500 CFA 7,000 CFA","9,500 CFA",https://cdn-jgmnn.nitrocdn.com/IeLUGrClUJaCmTO...,https://kevajo.com/boutique/eveline-cosmetics/...
2,2024-01-26,White Prestige 4D – Crème Blanchissante Pour l...,"5,000 CFA 3,500 CFA","5,000 CFA",https://cdn-jgmnn.nitrocdn.com/IeLUGrClUJaCmTO...,https://kevajo.com/boutique/eveline-cosmetics/...
3,2024-01-26,LUXURY EXPERT 24K GOLD – Lait Corporel Nourris...,"6,000 CFA","6,000 CFA",https://cdn-jgmnn.nitrocdn.com/IeLUGrClUJaCmTO...,https://kevajo.com/boutique/eveline-cosmetics/...
4,2024-01-26,HYALURON CLINIC B5 – Crème Pour Le Contour Des...,"3,500 CFA","3,500 CFA",https://cdn-jgmnn.nitrocdn.com/IeLUGrClUJaCmTO...,https://kevajo.com/boutique/eveline-cosmetics/...
...,...,...,...,...,...,...
845,2024-01-26,Chargeur rapide Micro USB,"3,600 CFA","3,600 CFA",https://cdn-jgmnn.nitrocdn.com/IeLUGrClUJaCmTO...,https://kevajo.com/boutique/telephones-et-tabl...
846,2024-01-26,Balance électronique portable avec écran LCD n...,"10,000 CFA 5,500 CFA","10,000 CFA",https://cdn-jgmnn.nitrocdn.com/IeLUGrClUJaCmTO...,https://kevajo.com/boutique/electronique/acces...
847,2024-01-26,Minister Of Oud – Oud Royal – Extrait De Parfu...,"35,000 CFA 13,500 CFA","35,000 CFA",https://cdn-jgmnn.nitrocdn.com/IeLUGrClUJaCmTO...,https://kevajo.com/boutique/beaute-et-hygiene/...
848,2024-01-26,Casque Bluetooth Multifonction P47 – Noir,"12,000 CFA 4,000 CFA","12,000 CFA",https://cdn-jgmnn.nitrocdn.com/IeLUGrClUJaCmTO...,https://kevajo.com/boutique/telephones-et-tabl...


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def extract_text(element, tag_name=None):
    if element and tag_name:
        tag = element.find(tag_name)
        return tag.text.strip() if tag else ""
    return ""

def clean_text(text):
    return text.replace('\r\n', '').replace('\xa0', '')

def scrape_kevajo_page(page_urls):
    product_data_list = []
    
    for page_url in page_urls:
        try:
            response = requests.get(page_url, timeout=500)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Erreur de connexion à {page_url} : {e}")
            continue

        soup = BeautifulSoup(response.text, 'html.parser')

        # Extraction des éléments de la page
        breadcrumbs = [a.text.strip() for a in soup.select('.woocommerce-breadcrumb a')]
        product_title = extract_text(soup.find('h1', class_='product_title'))
        

        # Extraction de la description
        description_tag = soup.find('div', class_='jt-description-content-wrapper')
        description = description_tag.find('p').text.strip() if description_tag else None
        url_lien=url

        attributes = {}
        attribute_rows = soup.select('.woocommerce-product-attributes tr')
        for row in attribute_rows:
            label = row.find('th').text.strip()
            value = row.find('td').text.strip()
            attributes[label] = value

        data = {
            'N_ordre': 'index',# Modify as needed
            'code_site': url_lien,  # Modify as needed
            'Code_ID_PE': 'YourCodeIDPE',  # Modify as needed
            'date de collecte': datetime.now().strftime('%Y-%m-%d'),
            'Breadcrumbs': breadcrumbs,
            'ProductTitle': product_title,
            'Attributes': attributes,
        }

        product_data_list.append(data)

    return pd.DataFrame(product_data_list)

# Liste des URLs de pages produits
page_urls = list(df_kevajo['URL'])

# Scrape des détails de chaque page
df_product_details = scrape_kevajo_page(page_urls)

# Assuming 'Attributes' column contains dictionaries
df_product_details['Quantité'] = df_product_details['Attributes'].apply(lambda x: x.get('Poids', ''))
df_product_details['Caractéristiques du produit'] = df_product_details['Attributes'].apply(lambda x: x.get('Marque', ''))

# Extract 'Poids', 'Unite', and 'Marque' columns from the 'Poids' column
df_product_details[['Quantité', 'Unite']] = df_product_details['Quantité'].str.extract(r"([0-9.]+)\s*([a-zA-Z]+)")

# Drop the original 'Poids' column
#df.drop('Poids', axis=1, inplace=True)

# Display the modified DataFrame
df_product_details


In [ ]:
df_kevajo['N_ordre'] = list(df_product_details["N_ordre"])
df_kevajo['code_site'] = list(df_product_details["code_site"])
df_kevajo['Code_ID_PE'] = list(df_product_details["Code_ID_PE"])
df_kevajo['date de collecte'] = list(df_product_details["date de collecte"])
df_kevajo['Caractéristiques du produit'] = list(df_product_details['Caractéristiques du produit'])
df_kevajo['Quantité'] = list(df_product_details["Quantité"])
df_kevajo['unite de mesure'] = list(df_product_details["Unite"])
df_kevajo['Intitule'] = list(df_product_details["Breadcrumbs"])
df_kevajo['Entreprise'] = list(df_product_details["ProductTitle"])

df_kevajo['Attributes'] = list(df_product_details["Attributes"])
df_kevajo

In [ ]:
# Réorganiser les colonnes selon vos besoins
df_kevajo = df_kevajo[[
    'N_ordre', 'code_site', 'Code_ID_PE', 'date de collecte', 'Libellé du produit', 'Caractéristiques du produit',
    'unite de mesure', 'Quantité', "Prix du produit"]]
df_kevajo

# Auchan

In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

# Liste des URLs
urls = [ "https://www.auchan.ci/les-indispensables/1J56N7JC/cp",
      "https://www.auchan.ci/mes-courses/gel-douche-et-bain/UJ3B2N7N",
     "https://www.auchan.ci/mes-courses/parapharmacie/18PEAELE", 
      "https://www.auchan.ci/mes-courses/cuisine-et-arts-de-la-table/1X78GLOL",
      "https://www.auchan.ci/selection-promos/1RVIB7FC/cp",
      "https://www.auchan.ci/selection-hygiene-beaute/CQQZ2P9E/cp",
      "https://www.auchan.ci/mes-courses/tampons/C3Z06PLP",
      "https://www.auchan.ci/mes-courses/fromages/CV5SB7BU",
      "https://www.auchan.ci/mes-courses/produits-laitiers-oeufs-fromages/UNBA9591",
      "https://www.auchan.ci/mes-courses/bieres-et-cidres/UR7QGB7B",
      "https://www.auchan.ci/mes-courses/viandes-charcuterie-et-poissons/COEBIN7N",
      "https://www.auchan.ci/mes-courses/produits-laitiers-oeufs-fromages/UNBA9591",
      "https://www.auchan.ci/mes-courses/fruits-et-legumes/CH0J2N7N",
      "https://www.auchan.ci/mes-courses/conserves-et-plats-cuisines/UBO4Q0Q1",
      "https://www.auchan.ci/mes-courses/rechercher?q=riz&queryId=a54ef1df64907375ef73a3313b5e2cce",
      "https://www.auchan.ci/mes-courses/hygiene-soins-homme/UED6QWQ1",
      "https://www.auchan.ci/mes-courses/thes-et-infusions/1JTX6N7N",
      "https://www.auchan.ci/mes-courses/fromages-rapes-et-a-cuisiner/1KJ6N5NC",
      "https://www.auchan.ci/mes-courses/dentifrices-et-bains-de-bouche/19D0SPBP",
    "https://www.auchan.ci/mes-courses/brioches-et-pains-de-mie/1LQB4N7N",
    "https://www.auchan.ci/mes-courses/soins-du-visage-et-maquillage/CX99MLOL",
    "https://www.auchan.ci/mes-courses/chocolat-en-poudre/1RB9SB7B",
    "https://www.auchan.ci/mes-courses/boissons-gazeuses-et-sirops/175K2PLP",
    "https://www.auchan.ci/mes-courses/bieres-et-cidres/UR7QGB7B",
    "https://www.auchan.ci/mes-courses/bonbons/1RNO6B7B",
    "https://www.auchan.ci/mes-courses/legumes-et-fruits/CRPYAB7B",
    "https://www.auchan.ci/mes-courses/cafes-moulus-grains-et-chicorees/1HRRAN7N",
    "https://www.auchan.ci/mes-courses/accessoires-de-cuisine-salle-de-bains-et-wc/U0DBAPBP",
    "https://www.auchan.ci/mes-courses/vins/CP7F2B7B",
    "https://www.auchan.ci/mes-courses/eaux/19Z32PBP",
    "https://www.auchan.ci/mes-courses/produits-laitiers-oeufs-fromages/UNBA9591",
    "https://www.auchan.ci/mes-courses/chiens/UZZ9GLOL",
      "https://www.auchan.ci/mes-courses/javel-et-produits-multi-usages/19XT4PBP",
      "https://www.auchan.ci/mes-courses/corps/CZ02LOLU",
    "https://www.auchan.ci/mes-courses/jambons-et-saucissons-de-porc/UVZP6B7B",
    "https://www.auchan.ci/mes-courses/farines/1J0Y2N7N",
    "https://www.auchan.ci/mes-courses/papiers-toilette/UVWF2B7B",
      "https://www.auchan.ci/mes-courses/aperitif/1EN9IQWQ",
      "https://www.auchan.ci/mes-courses/cuisine/CTPXSB5B",
    "https://www.auchan.ci/mes-courses/charcuterie/13NXAPLP",
    "https://www.auchan.ci/les-nouveautes/CYXMB7NU/cp",
    "https://www.auchan.ci/mes-courses/boissons/UFRY2VYV",
    "https://www.auchan.ci/mes-courses/savons/UENW2QWQ",
    "https://www.auchan.ci/mes-courses/cereales/UPNJ4B7B",
      "https://www.auchan.ci/mes-courses/couches-et-couches-culottes/1ZOYSLOL",
    "https://www.auchan.ci/mes-courses/cafes/U0FDIPBP",
    "https://www.auchan.ci/mes-courses/soins-des-cheveux/UZZNGLOL",
    "https://www.auchan.ci/mes-courses/laits-et-boissons-lactees/C7NZMPLP",
    "https://www.auchan.ci/mes-courses/serviettes-hygieniques/1EOXAQWQ",
    "https://www.auchan.ci/special-de-2000fcfa/C5ESPLRU/cp",
    "https://www.auchan.ci/mes-courses/laits-concentres/UYEN2B7B",
      "https://www.auchan.ci/mes-courses/oeufs/CELZGQWQ",
    "https://www.auchan.ci/mes-courses/bles-semoules-et-cereales/UQQV6P9P",
    "https://www.auchan.ci/mes-courses/couches-lingettes-et-cotons/UWO92P9P",
    "https://www.auchan.ci/mes-courses/boucherie/CW9WSP9P",
    "https://www.auchan.ci/mes-courses/plats-cuisines/1P972B7B",
    "https://www.auchan.ci/mes-courses/biscuits-secs-sables-et-assortiments/10TMPBPC",
    "https://www.auchan.ci/mes-courses/epicerie-sucree/CDWAVYVU",
    "https://www.auchan.ci/mes-courses/poulet-et-coq/13QE2PLP",
    "https://www.auchan.ci/mes-courses/produits-nettoyants/1R8P4B7B",
      "https://www.auchan.ci/epicerie-salee/CBBO2Q03/cp",
    "https://www.auchan.ci/mes-courses/yaourts-et-desserts/UEX02QWQ",
    "https://www.auchan.ci/mes-courses/cremes-et-soins/1H0R6N7N",
    "https://www.auchan.ci/mes-courses/hygiene-et-beaute/CPOSB7BU",
    "https://www.auchan.ci/selection-boissons/1W94P9EC/cp",
    "https://www.auchan.ci/mes-courses/liqueurs-et-spiritueux/15K9GPLP",
    "https://www.auchan.ci/mes-courses/conserves-et-plats-cuisines/UBO4Q0Q1", 
    "https://www.auchan.ci/mes-courses/cremes-et-soins/1H0R6N7N",
    "https://www.auchan.ci/mes-courses/vaisselle-jetable/18LR6ELE",
    "https://www.auchan.ci/mes-courses/lessives-repassage-et-soin-du-linge/1KOW2N5N",
    "https://www.auchan.ci/mes-courses-avec-livraison/1V3AB7FC/cp",
    "https://www.auchan.ci/mes-courses-du-mois/CVYIB7FU/cp",
    "https://www.auchan.ci/mes-courses/ketchup-mayonnaise-moutarde-et-sauces/UO8ZSN7N",
    "https://www.auchan.ci/mes-courses/panaches/CRXB4B7B",
    "https://www.auchan.ci/mes-courses/gel-douche-et-bain/UEKIQWQ1",
      "https://www.auchan.ci/mes-courses/oeufs-beurres-et-creme/1LOLAN7N",
    "https://www.auchan.ci/mes-courses/produits-menagers-et-accessoires-de-la-maison/C3QIPLPU",
    "https://www.auchan.ci/mes-courses/cremes-fraiches/CBOBGQ0Q",
    "https://www.auchan.ci/mes-courses/ustensiles-de-cuisine/1Z7JILOL",
    "https://www.auchan.ci/mes-courses/dosettes-et-capsules/C3DR2PLP",
    "https://www.auchan.ci/mes-courses/wc-et-canalisations/CHHW6N7N",
    "https://www.auchan.ci/mes-courses/cereales-et-petit-dejeuner-bebe/1Y9Z6B7B",
    "https://www.auchan.ci/mes-courses/colas-et-boissons-gazeuses/1NOJI959",
    "https://www.auchan.ci/mes-courses/volaille-gibier-lapin-et-autres/UZ9ZSLOL",
    "https://www.auchan.ci/mes-courses/plats-cuisines-en-conserve/1YLVIB7B",
    "https://www.auchan.ci/mes-courses/shampoings/URVNSB7B",
    "https://www.auchan.ci/promos-can/U5DH4PLR/cp",
      "https://www.auchan.ci/mes-courses/surgeles/UYB4B7B1",
      "https://www.auchan.ci/mes-courses/beurres-margarines-et-autres/UJBB6N7N",
      "https://www.auchan.ci/mes-courses/eaux-minerales/C7ORAPLP",
      "https://www.auchan.ci/mes-courses/bebe/U8P06ELE",
    "https://www.auchan.ci/mes-courses/huiles-epices-condiments/CQQDSP9P",
    "https://www.auchan.ci/mes-courses/pates/10EAPBPC",
    "https://www.auchan.ci/mes-courses/riz/1QH6P9PC",
    "https://www.auchan.ci/mes-courses/vinaigres-vinaigrettes-et-sauces-crudites/UZXQSLOL",
    "https://www.auchan.ci/mes-courses/fruits-et-legumes/CH0J2N7N",
    "https://www.auchan.ci/tout-a-moins-de-1000f/C5ESPLRU/cp",
    "https://www.auchan.ci/mes-courses/epicerie-salee/CENGQWQU",
    "https://www.auchan.ci/mes-courses/mouchoirs/CZVNGLOL",
    "https://www.auchan.ci/mes-courses/laits-en-poudre/C0PE2PBP",
      
     ] 

# Configuration du navigateur Selenium
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
driver = webdriver.Chrome(options=options)

# Initialiser une liste pour stocker les données
data_list = []

# Parcourir la liste des URLs
for url in urls:
    driver.get(url)
    # Attendre que la page se charge complètement
    wait = WebDriverWait(driver, 1200)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.js-jt-product-card')))

    # Parser le contenu HTML avec BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Sélectionner tous les produits sur la page
    products = soup.select('.js-jt-product-card')

    # Boucle sur chaque produit
    for product in products:
        # Extraire les informations nécessaires
        product_pid = product['cy-product-pid']
        product_title = product.select_one('.js-title-line').text.strip()
        product_brand = product.select_one('.js-brand-line').text.strip()
        product_image = product.select_one('.js-image-line')['data-src']
        product_price = product.select_one('.js-price-line').text.strip()
        product_old_price = product.select_one('.js-wasPrice-line')

        url_lien = url

        # Vérifier si le prix d'origine existe
        if product_old_price:
            product_old_price = product_old_price.text.strip()
        else:
            product_old_price = None

        # Ajouter les données à la liste
        data_list.append({
            'N_ordre': product_pid,
            'Libellé': product_title,
            'code_site': url_lien,
            'Code_ID_PE': product_brand,
            'Product Image': product_image,
            'Prix du produit': product_price,
            'Product Old Price': product_old_price,
            'date de collecte': datetime.now().strftime('%Y-%m-%d'),
        })

        # Cliquer sur l'élément pour obtenir plus de détails
        product_link = product.select_one('.js-product-anchor')
        if product_link and product_link.has_attr('href'):
            product_details_url = product_link['href']

            # Ouvrir un nouvel onglet pour charger la page de détails
            driver.execute_script(f"window.open('{product_details_url}', '_blank');")

            # Passer au nouvel onglet
            driver.switch_to.window(driver.window_handles[1])

            # Attendre que la page de détails se charge complètement
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.jt-breadcrumb-title-ellipsis span:last-child')))

            # Parser le contenu HTML de la page de détails
            detail_soup = BeautifulSoup(driver.page_source, 'html.parser')

            # Extraire des informations supplémentaires de la page de détails
            product_description = detail_soup.select_one('.jt-description-content-wrapper p').text.strip()

            # Ajouter les informations supplémentaires à la liste
            data_list[-1]['Caractéristiques du produit'] = product_description

            # Fermer le nouvel onglet
            driver.close()

            # Revenir à l'onglet principal
            driver.switch_to.window(driver.window_handles[0])

# Fermer le navigateur Selenium
driver.quit()

# Créer un DataFrame à partir de la liste
df_auchan = pd.DataFrame(data_list)

# Imprimer le DataFrame
df_auchan


,N_ordre,Libellé,code_site,Code_ID_PE,Product Image,Prix du produit,Product Old Price,date de collecte,Caractéristiques du produit
0,KOOEIN55,Jus d'ananas 33cl,https://www.auchan.ci/les-indispensables/1J56N...,IVORIO,image:jt/p/KOOEIN55/LOOB2N7D_5999991071418.png,460 FCFA,490 FCFA,2024-01-26,"Une boisson rafraîchissante sans conservateur,..."
1,D3JF4VYY,"Bière aromatisée à la tequilla 5,5% 33cl",https://www.auchan.ci/les-indispensables/1J56N...,BOOSTER,image:jt/p/D3JF4VYY/QLJZIP9D_photo201222104509...,450 FCFA,,2024-01-26,"Produite localement, la bière tequilla se dist..."
2,YWTO4B77,Lait demi-écrémé 50cl,https://www.auchan.ci/les-indispensables/1J56N...,CANDIA,image:jt/p/YWTO4B77/WEQTMP9D_3ol3mplo_1700111-...,590 FCFA,,2024-01-26,"GrandLait demi-écrémé, le plaisir d'un lait de..."
3,QZO7SP99,Huile végétale locale 5L,https://www.auchan.ci/les-indispensables/1J56N...,DINOR,image:jt/p/QZO7SP99/Y9XW6B7L_1000626.jpg,5750 FCFA,5790 FCFA,2024-01-26,L'huile Dinor est une Huile qui fait la fierté...
4,WQE8GP99,Bière 33cl,https://www.auchan.ci/les-indispensables/1J56N...,DOPPEL,image:jt/p/WQE8GP99/N3D5G95Q_doppel-2020-e1605...,390 FCFA,425 FCFA,2024-01-26,Entrez dans l’univers des bières brunes qui dé...
...,...,...,...,...,...,...,...,...,...
3519,ON83SN77,Lait en poudre fortigrow 800g,https://www.auchan.ci/mes-courses/laits-en-pou...,NIDO,image:jt/p/ON83SN77/OYT3IN7Y_1004424.jpg,7225 FCFA,7490 FCFA,2024-01-26,"Le lait en poudre NIDO Frotigrow, en boîte de ..."
3520,9XOKSPBB,Lait entier fortigrow 900g,https://www.auchan.ci/mes-courses/laits-en-pou...,NIDO,image:jt/p/9XOKSPBB/3XO04PLO_8717896009292.jpg,7650 FCFA,,2024-01-26,La marque propose une gamme complète de laits ...
3521,TYEHSB55,Lait en poudre instantané 25g,https://www.auchan.ci/mes-courses/laits-en-pou...,LEBON,image:jt/p/TYEHSB55/OK90GN7Y_1001523.png,90 FCFA,,2024-01-26,Offrez-vous une tasse de lait onctueux en un c...
3522,QX3NGP99,Sachet café 40g,https://www.auchan.ci/mes-courses/laits-en-pou...,TOP LAIT,image:jt/p/QX3NGP99/3LOR4PLT_mon-projet-_40_.png,100 FCFA,150 FCFA,2024-01-26,"Cafés, Boissons instantanées, Café au lait suc..."


In [15]:
import re
import pandas as pd

# Fonction pour extraire le nom du produit, la quantité et l'unité de mesure
def extract_info(product_title):
    # Utiliser une expression régulière pour extraire les informations
    match = re.match(r'(?P<Libellé_du_produit>.*?)(?P<Quantité>\d+)(?P<Unité_de_mesure>[a-zA-Z]+)', product_title)
    
    # Vérifier si la correspondance a été trouvée
    if match:
        return match.group('Libellé_du_produit').strip(), match.group('Quantité'), match.group('Unité_de_mesure')
    else:
        return None, None, None

# Appliquer la fonction sur la colonne "Product Title"
df_auchan[['Libellé du produit', 'Quantité', 'unite de mesure']] = df_auchan["Libellé"].apply(extract_info).apply(pd.Series)

# Imprimer le DataFrame mis à jour
df_auchan


,N_ordre,Libellé,code_site,Code_ID_PE,Product Image,Prix du produit,Product Old Price,date de collecte,Caractéristiques du produit,Libellé du produit,Quantité,unite de mesure
0,KOOEIN55,Jus d'ananas 33cl,https://www.auchan.ci/les-indispensables/1J56N...,IVORIO,image:jt/p/KOOEIN55/LOOB2N7D_5999991071418.png,460 FCFA,490 FCFA,2024-01-26,"Une boisson rafraîchissante sans conservateur,...",Jus d'ananas,33,cl
1,D3JF4VYY,"Bière aromatisée à la tequilla 5,5% 33cl",https://www.auchan.ci/les-indispensables/1J56N...,BOOSTER,image:jt/p/D3JF4VYY/QLJZIP9D_photo201222104509...,450 FCFA,,2024-01-26,"Produite localement, la bière tequilla se dist...","Bière aromatisée à la tequilla 5,5%",33,cl
2,YWTO4B77,Lait demi-écrémé 50cl,https://www.auchan.ci/les-indispensables/1J56N...,CANDIA,image:jt/p/YWTO4B77/WEQTMP9D_3ol3mplo_1700111-...,590 FCFA,,2024-01-26,"GrandLait demi-écrémé, le plaisir d'un lait de...",Lait demi-écrémé,50,cl
3,QZO7SP99,Huile végétale locale 5L,https://www.auchan.ci/les-indispensables/1J56N...,DINOR,image:jt/p/QZO7SP99/Y9XW6B7L_1000626.jpg,5750 FCFA,5790 FCFA,2024-01-26,L'huile Dinor est une Huile qui fait la fierté...,Huile végétale locale,5,L
4,WQE8GP99,Bière 33cl,https://www.auchan.ci/les-indispensables/1J56N...,DOPPEL,image:jt/p/WQE8GP99/N3D5G95Q_doppel-2020-e1605...,390 FCFA,425 FCFA,2024-01-26,Entrez dans l’univers des bières brunes qui dé...,Bière,33,cl
...,...,...,...,...,...,...,...,...,...,...,...,...
3519,ON83SN77,Lait en poudre fortigrow 800g,https://www.auchan.ci/mes-courses/laits-en-pou...,NIDO,image:jt/p/ON83SN77/OYT3IN7Y_1004424.jpg,7225 FCFA,7490 FCFA,2024-01-26,"Le lait en poudre NIDO Frotigrow, en boîte de ...",Lait en poudre fortigrow,800,g
3520,9XOKSPBB,Lait entier fortigrow 900g,https://www.auchan.ci/mes-courses/laits-en-pou...,NIDO,image:jt/p/9XOKSPBB/3XO04PLO_8717896009292.jpg,7650 FCFA,,2024-01-26,La marque propose une gamme complète de laits ...,Lait entier fortigrow,900,g
3521,TYEHSB55,Lait en poudre instantané 25g,https://www.auchan.ci/mes-courses/laits-en-pou...,LEBON,image:jt/p/TYEHSB55/OK90GN7Y_1001523.png,90 FCFA,,2024-01-26,Offrez-vous une tasse de lait onctueux en un c...,Lait en poudre instantané,25,g
3522,QX3NGP99,Sachet café 40g,https://www.auchan.ci/mes-courses/laits-en-pou...,TOP LAIT,image:jt/p/QX3NGP99/3LOR4PLT_mon-projet-_40_.png,100 FCFA,150 FCFA,2024-01-26,"Cafés, Boissons instantanées, Café au lait suc...",Sachet café,40,g


In [16]:
# Réorganiser les colonnes selon vos besoins
df_auchan = df_auchan[[
    'N_ordre', 'code_site', 'Code_ID_PE', 'date de collecte', 'Libellé du produit', 'Caractéristiques du produit',
    'unite de mesure', 'Quantité', "Prix du produit"]]
df_auchan

,N_ordre,code_site,Code_ID_PE,date de collecte,Libellé du produit,Caractéristiques du produit,unite de mesure,Quantité,Prix du produit
0,KOOEIN55,https://www.auchan.ci/les-indispensables/1J56N...,IVORIO,2024-01-26,Jus d'ananas,"Une boisson rafraîchissante sans conservateur,...",cl,33,460 FCFA
1,D3JF4VYY,https://www.auchan.ci/les-indispensables/1J56N...,BOOSTER,2024-01-26,"Bière aromatisée à la tequilla 5,5%","Produite localement, la bière tequilla se dist...",cl,33,450 FCFA
2,YWTO4B77,https://www.auchan.ci/les-indispensables/1J56N...,CANDIA,2024-01-26,Lait demi-écrémé,"GrandLait demi-écrémé, le plaisir d'un lait de...",cl,50,590 FCFA
3,QZO7SP99,https://www.auchan.ci/les-indispensables/1J56N...,DINOR,2024-01-26,Huile végétale locale,L'huile Dinor est une Huile qui fait la fierté...,L,5,5750 FCFA
4,WQE8GP99,https://www.auchan.ci/les-indispensables/1J56N...,DOPPEL,2024-01-26,Bière,Entrez dans l’univers des bières brunes qui dé...,cl,33,390 FCFA
...,...,...,...,...,...,...,...,...,...
3519,ON83SN77,https://www.auchan.ci/mes-courses/laits-en-pou...,NIDO,2024-01-26,Lait en poudre fortigrow,"Le lait en poudre NIDO Frotigrow, en boîte de ...",g,800,7225 FCFA
3520,9XOKSPBB,https://www.auchan.ci/mes-courses/laits-en-pou...,NIDO,2024-01-26,Lait entier fortigrow,La marque propose une gamme complète de laits ...,g,900,7650 FCFA
3521,TYEHSB55,https://www.auchan.ci/mes-courses/laits-en-pou...,LEBON,2024-01-26,Lait en poudre instantané,Offrez-vous une tasse de lait onctueux en un c...,g,25,90 FCFA
3522,QX3NGP99,https://www.auchan.ci/mes-courses/laits-en-pou...,TOP LAIT,2024-01-26,Sachet café,"Cafés, Boissons instantanées, Café au lait suc...",g,40,100 FCFA


In [20]:


# Supposons que df_auchan, df_ivoirshop et df_kevajo sont vos trois DataFrames

# Concaténation verticale (ajout des lignes)
Web_scraping_Auchan_Kevajo_Ivoirshop = pd.concat([df_ivoirshop,df_kevajo, df_auchan], ignore_index=True)

# Si vous ne souhaitez pas réinitialiser les index, vous pouvez laisser ignore_index=False

# Afficher le résultat
Web_scraping_Auchan_Kevajo_Ivoirshop


,N_ordre,code_site,Code_ID_PE,date de collecte,Libellé du produit,Caractéristiques du produit,unite de mesure,Quantité,Prix du produit,Caratéristiques du produit,Prix Réel
0,index,https://kevajo.com/,YourCodeIDPE,2024-01-26,Rouge à Lèvres Liquide Mat Et Crayon à Lèvres ...,Eveline Cosmetics,kg,0.2,"2,000 CFA",NaN,NaN
1,index,https://kevajo.com/,YourCodeIDPE,2024-01-26,SLIM EXTREME 4D – Sérum Surconcentré Réducteur...,Eveline Cosmetics,kg,0.3,"9,500 CFA 7,000 CFA",NaN,NaN
2,index,https://kevajo.com/,YourCodeIDPE,2024-01-26,White Prestige 4D – Crème Blanchissante Pour l...,Eveline Cosmetics,kg,0.1,"5,000 CFA 3,500 CFA",NaN,NaN
3,index,https://kevajo.com/,YourCodeIDPE,2024-01-26,LUXURY EXPERT 24K GOLD – Lait Corporel Nourris...,Eveline Cosmetics,kg,0.425,"6,000 CFA",NaN,NaN
4,index,https://kevajo.com/,YourCodeIDPE,2024-01-26,HYALURON CLINIC B5 – Crème Pour Le Contour Des...,Eveline Cosmetics,kg,0.08,"3,500 CFA",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3823,ON83SN77,https://www.auchan.ci/mes-courses/laits-en-pou...,NIDO,2024-01-26,Lait en poudre fortigrow,"Le lait en poudre NIDO Frotigrow, en boîte de ...",g,800,7225 FCFA,NaN,NaN
3824,9XOKSPBB,https://www.auchan.ci/mes-courses/laits-en-pou...,NIDO,2024-01-26,Lait entier fortigrow,La marque propose une gamme complète de laits ...,g,900,7650 FCFA,NaN,NaN
3825,TYEHSB55,https://www.auchan.ci/mes-courses/laits-en-pou...,LEBON,2024-01-26,Lait en poudre instantané,Offrez-vous une tasse de lait onctueux en un c...,g,25,90 FCFA,NaN,NaN
3826,QX3NGP99,https://www.auchan.ci/mes-courses/laits-en-pou...,TOP LAIT,2024-01-26,Sachet café,"Cafés, Boissons instantanées, Café au lait suc...",g,40,100 FCFA,NaN,NaN
